In [1]:
from collect_data import *

In [2]:
meg_path = '/data01/data/MEG'
os.listdir(meg_path)

['index.html?zip=',
 'README.txt',
 'participants.tsv',
 'participants.json',
 'dataset_description.json',
 'sub-11',
 'sub-10',
 'sub-09',
 'sub-08',
 'sub-07',
 'sub-06',
 'sub-05',
 'sub-04',
 'sub-03',
 'sub-02',
 'sub-01',
 'stimuli',
 'meg.gz']

In [3]:
stimuli_path = meg_path + '/stimuli/audio'
wav_files_duration = {}

for filename in os.listdir(stimuli_path):
    if filename.endswith('.wav'): 
        file_path = os.path.join(stimuli_path, filename)
        y, sr = librosa.load(file_path, sr=None)
        duration = librosa.get_duration(y=y, sr=sr)
        wav_files_duration[filename.rstrip('.wav')] = duration
print('WAVE FILES DURATION: ',wav_files_duration)

wav_files_without_numb = {'lw1': 0, 'cable_spool_fort': 1, 'easy_money': 2, 'the_black_willow': 3}
wav_list_without_numb = list(wav_files_without_numb.keys())
print('WAVE FILES WITH\ NUMBERS: ',wav_files_without_numb)



WAVE FILES DURATION:  {'lw1': 318.1481632653061, 'lw1_3': 53.072426303854876, 'lw1_2': 93.7512925170068, 'lw1_1': 75.44276643990929, 'lw1_0': 95.88167800453515, 'easy_money_7': 129.30312925170068, 'easy_money_6': 123.42730158730159, 'easy_money_4': 163.68920634920636, 'easy_money_5': 142.36866213151927, 'easy_money_3': 141.97519274376418, 'easy_money_0': 143.12875283446712, 'easy_money_2': 180.43360544217688, 'easy_money_1': 104.10834467120182, 'cable_spool_fort_4': 123.26390022675737, 'cable_spool_fort_5': 54.69605442176871, 'cable_spool_fort_3': 105.82820861678005, 'cable_spool_fort_2': 137.65614512471655, 'cable_spool_fort_1': 134.8726984126984, 'cable_spool_fort_0': 100.59777777777778, 'the_black_willow_9': 188.23668934240362, 'the_black_willow_8': 142.4142403628118, 'the_black_willow_7': 172.399410430839, 'the_black_willow_6': 128.21628117913832, 'the_black_willow_5': 81.94331065759637, 'the_black_willow_4': 110.45659863945578, 'the_black_willow_2': 125.23859410430839, 'the_black_

In [4]:
bids_path = mne_bids.BIDSPath(
    subject = '01',
    session = '0', 
    task = '0', 
    datatype = "meg", 
    root = meg_path,
)
raw = mne_bids.read_raw_bids(bids_path) 
raw.load_data().filter(0.5, 30.0, n_jobs=1)   
df = raw.annotations.to_data_frame()
df = pd.DataFrame(df.description.apply(eval).to_list())
raw = raw.pick_types(
    meg=True, misc=False, eeg=False, eog=False, ecg=False
)
print('CHECK_RAW', raw)


Extracting SQD Parameters from /data01/data/MEG/sub-01/ses-0/meg/sub-01_ses-0_task-0_meg.con...
Creating Raw.info structure...
Setting channel info structure...
Creating Info structure...
Ready.
Reading events from /data01/data/MEG/sub-01/ses-0/meg/sub-01_ses-0_task-0_events.tsv.
Reading channel info from /data01/data/MEG/sub-01/ses-0/meg/sub-01_ses-0_task-0_channels.tsv.
The stimulus channel "STI 014" is present in the raw data, but not included in channels.tsv. Removing the channel.
Reading 0 ... 395999  =      0.000 ...   395.999 secs...


/tmp/ipykernel_656130/2560004595.py:8: RuntimeWarning: The unit for channel(s) MISC 001, MISC 002, MISC 003, MISC 004, MISC 005, MISC 006, MISC 007, MISC 008, MISC 009, MISC 010, MISC 011, MISC 012, MISC 013, MISC 014, MISC 015, MISC 016, MISC 017, MISC 018, MISC 019, MISC 020, MISC 021, MISC 022, MISC 023, MISC 024, MISC 025, MISC 026, MISC 027, MISC 028, MISC 029, MISC 030, MISC 031, MISC 032 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 6601 samples (6.601 s)

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
CHECK_RAW <RawKIT | sub-01_ses-0_task-0_meg.con, 208 x 396000 (396.0 s), ~628.6 MB, data loaded>


In [5]:
x = raw.get_data()
sampling_freq = raw.info["sfreq"] 
print('RAW_SHAPE: ', x.shape)

RAW_SHAPE:  (208, 396000)


In [6]:
get_epochs = get_epochs(raw, 0.0)
epoch_spectr = get_meg_from_raw_epochs(get_epochs)
meg_spectr = get_meg_spectrogram(epoch_spectr)
print('MEG_SPECTR_SHAPE: ', meg_spectr.shape)


Adding metadata with 18 columns
180 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 180 events and 3201 original time points ...
0 bad epochs dropped
Applying baseline correction (mode: mean)


/home/matteoc/spect-to-meg/collect_data.py:97: ComplexWarning: Casting complex values to real discards the imaginary part
  spectrograms[i, j] = d


MEG_SPECTR_SHAPE:  torch.Size([180, 208, 257, 26])


In [ ]:
for i in range(meg_spectr.shape[0]):
    plt.figure(figsize=(10, 4))
    #librosa.display.specshow(meg_spectr[i][1].numpy(), sr=1000, x_axis='time', y_axis='log')
    plt.imshow(meg_spectr[i][1].numpy())
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram')
    plt.tight_layout()
    plt.show()
    break

In [7]:
meg_spectr.sum()

tensor(1.9872e-07, dtype=torch.float64)

In [15]:
audio_path = stimuli_path + '/' + wav_list_without_numb[0] + '_0' + '.wav'
print('AUDIO_PATH: ',audio_path)
audio, sr = librosa.load(audio_path, sr=sampling_audio)
tot_length_audio = audio.shape[0] / sr
print('AUDIO_LENGTH_SAMPLES: ',audio.shape)
print('AUDIO_LENGTH_SECONDS: ',tot_length_audio)

AUDIO_PATH:  /data01/data/MEG/stimuli/audio/lw1_0.wav
AUDIO_LENGTH_SAMPLES:  (1534107,)
AUDIO_LENGTH_SECONDS:  95.8816875


In [16]:
audio_spectr = get_audio_spectrogram(audio_path, get_epochs)
print('AUDIO_SPECTR_SHAPE: ', audio_spectr.shape)

/home/matteoc/spect-to-meg/collect_data.py:108: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  y_db = librosa.amplitude_to_db(librosa.stft(y, n_fft=n_fft, hop_length=hop_length), ref=np.max)


AUDIO_SPECTR_SHAPE:  torch.Size([180, 257, 376])
